In [1]:
# Exercise 2 SCIPHER

In [2]:
import itertools
import hashlib

from helper import *

In [3]:
params = load_params('params.txt')
params_Q2 = load_params('Q2-hash.txt')

Q2a_p   = int(params['Q2a_p'])
Q2a_g   = int(params['Q2a_g'])
Q2a_sk  = int(params['Q2a_sk'])
Q2a_C   = params['Q2a_C']
Q2b_p   = params['Q2b_p']
Q2b_g   = params['Q2b_g']
Q2b_pk  = params['Q2b_pk']
Q2b_C1  = params['Q2b_C1']
Q2b_KEY = params['Q2b_KEY']

Q2b_C2hash = params_Q2['Q2b_C2hash']

In [4]:
# Question 2.1 : Implements Dec(sk, C, pp) and find Q2a_C


def parse_cipher(C):
    c1_c2 = []
    for pairs_hex in C.split(';')[:-1]:
        c1_hex, c2_hex = pairs_hex.split(':')
        c1_c2.append((int(c1_hex, 16), int(c2_hex, 16)))
    return c1_c2


def enc(pk, m, pp):
    p, g = pp
    G = IntegerModRing(p)
    C = ''
    for i in range(len(m)):
        mi = m[i]
        r = int(G.random_element())
        c1 = Mod(power_mod(pk, r, p) * power_mod(g, ord(mi), p), p)
        c2 = power_mod(g, r, p)
        C += hex(c1)[2:] + ':' + hex(c2)[2:] + ';'
    return C

def dec(sk, C, pp):
    p, g = pp
    c1_c2 = parse_cipher(C)
    
    plaintext = ''
    for c1, c2 in c1_c2:
        gm = Mod(c1 * power_mod(c2, -sk, p), p)
        for i in range(65, 91):  # for i in range(0, 200):
            gi = power_mod(g, i, p)
            if gi == gm:
                plaintext += chr((i - ord('A')) % 26 + ord('A'))
    
    return plaintext
    
    
print('Q2a_m={}'.format(dec(Q2a_sk, Q2a_C, (Q2a_p, Q2a_g))))

Q2a_m=RECKLESSCACKLING


In [5]:
# Question 2.2


def caesar_char(cipher_c, shift):
    if cipher_c < 'A' or cipher_c > 'Z':
        return cipher_c
    else:
        return chr((ord(cipher_c) - ord('A') + shift) % 26 + ord('A'))
    
def vigenere(text, key):
    key_repeated = key * ceil(len(text) / len(key))
    return ''.join([caesar_char(text[index], ord('A') - ord(key_repeated[index])) for index in range(len(text))])


def SCIPHER(m, K):
    key = [chr(ord('A') + ord(digit) - ord('0')) for digit in K]
    return vigenere(m, key)


def find_solt(pk, C1, pp, K):
    p, g = pp
    c1_c2 = parse_cipher(C1)
    key_repeated = K * ceil(len(C1) / len(K))
    new_C = ''
    for i in range(len(c1_c2)):
        c1, c2 = c1_c2[i]
        shift = ord(key_repeated[i]) - ord('0')
        new_c1 = Mod(c1 * power_mod(g, shift, p), p)  # TODO check if shift or -shift
        new_C += hex(new_c1)[2:] + ':' + hex(c2)[2:] + ';'
    
    return new_C

new_C = find_solt(Q2b_pk, Q2b_C1, (int(Q2b_p), int(Q2b_g)), Q2b_KEY)
valid = hashlib.sha256(new_C.encode()).hexdigest() == Q2b_C2hash

print('Q2b_C2 hash is valid : {}'.format(valid))
print('Q2b_C2={}'.format(new_C))

Q2b_C2 hash is valid : True
Q2b_C2=be835db50d89b6829e45cbfe7ac3f5434fdbcc11a4e88e6593821e181bad57c53bbe8173e78b8e08fcc19b97eb89eae835eb99aa7863468f75198ffcf581c3772787039280d8e4bb43a3477ef235133362259ea56032e41028692ba656a89bea803371c89522d81b67266d0d5c0ead7032ae08d7ae37e7fcac8933fbbffe1175:d3b1f17be26c4782915a5c2e83824647c43c5ee1abb6e1c121930f55de48db985ddc853cfde1578662588c752cc41cf1460894cbe788a820f232f549c6fc2347a235779604fa82fe0090ca22bd508a35aa2e69eb5a17bee38e53639154f7b55a47c52c8234a189b88278bc8fa194634a17a7fc91d070e84b406949ae6b4321db;d0673a9526747b3a9a2eb3497ebc1b4b73cc6e43f6c8647d54b61156d2091b478ea2f9dd558868be14bfe47d282ccd52f154d0259a2cb6b68eafaa1e5601f3199bb0b428723dff2a5838754af4051566a47fc2844e14d669b014981ec19c7071b05230fda6db4e988a30cc512cee89d000a08025ea2b47421eb0226310ee2f14:95e83da1ee7c3fdfcefed1cb65c12ea07aa66fa8f0ab7f6c837bfc429b62975e6ea5f2aca4a34cb5edf5b389ca773274868cce94bf524ebb89f3f6e276cd9ca99bd393aee8603e66a94af6cbf6ea10da5fc64055d1d329955a2505e817f4c9c71a